In [1]:
import numpy as np
import xarray as xr
import pygmt

In [2]:
with open('real_input.txt', 'r') as fh:
    lines = [line.strip() for line in fh.readlines()]

letter_values = {letter: value for value, letter in enumerate('abcdefghijklmnopqrstuvwxyz')}
    
numx = len(lines[0])
numy = len(lines)
grid = np.zeros((numy, numx))

for x in np.arange(numx):
    for y in np.arange(numy-1, -1, -1):
        letter = lines[y][x]
        if letter == 'S':
            grid[y,x] = 0
            start = (y, x)
        elif letter == 'E':
            grid[y,x] = 26
            end = (y, x)
        elif letter in letter_values.keys():
            grid[y,x] = letter_values[letter]
        else:
            print("Invalid letter: ", letter)
    

In [3]:
# direct_distance_grid = np.zeros((numy, numx))
# rightmove_grid = np.zeros((numy, numx))
# leftmove_grid = np.zeros((numy, numx))
# upmove_grid = np.zeros((numy, numx))
# downmove_grid = np.zeros((numy, numx))
# for iy in range(numy):
#     ydist = iy-end[0]
#     for ix in range(numx):
#         xdist = ix-end[1]
#         direct_distance_grid[iy, ix] = np.sqrt(ydist*ydist+xdist*xdist)
#         rightmove_grid[iy, ix] = grid[iy,min(ix+1,numx-1)] <= grid[iy,ix] + 1
#         leftmove_grid[iy, ix] = grid[iy,max(ix-1,0)] <= grid[iy,ix] + 1
#         upmove_grid[iy, ix] = grid[min(iy+1,numy-1),ix] <= grid[iy,ix] + 1
#         downmove_grid[iy, ix] = grid[max(iy-1,0),ix] <= grid[iy,ix] + 1
# # distance_grid = np.max(distance_grid) - distance_grid

In [4]:
distance_grid = np.zeros((numy, numx))
distance_grid[:,:] = 1e33
visited_grid = np.zeros((numy, numx))
visited_grid[:,:] = 0
idx_x_grid = np.zeros((numy, numx), dtype='int')
idx_x_grid[:,:] = -1
idx_y_grid = np.zeros((numy, numx), dtype='int')
idx_y_grid[:,:] = -1

distance_grid[start] = 0
idx_y_grid[start] = start[0]
idx_x_grid[start] = start[1]

def valid_steps(grid, start):
    steps = []
    directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    for direction in directions:
        new_position = tuple(start + np.array(direction))
        if new_position[0] < 0 or new_position[0] > numy-1 or new_position[1] < 0 or new_position[1] > numx-1:
            # not in grid
            continue
        if grid[tuple(new_position)] > grid[tuple(start)] + 1:
            # too steep to climb without climbing gear
            continue
        steps.append(tuple(new_position))
    return steps


current = start
step = current
def traverse(grid, current, end):
    global visited_grid, distance_grid, idx_y_grid, idx_x_grid
    step = current
    if np.all(np.array(step) == np.array(end)):
        return True
    for step in valid_steps(grid, current):
        if distance_grid[current] + 1 < distance_grid[step]: # Update shortest path to this point
            distance_grid[step] = distance_grid[current] + 1
            idx_y_grid[step] = current[0]
            idx_x_grid[step] = current[1]
        if visited_grid[step] == 0:
            visited_grid[step] = 1
            traverse(grid, step, end)
            
traverse(grid, start, end)

In [5]:
path = []
current = end
while current != start:
    path.append(list(current))
    current = (idx_y_grid[current], idx_x_grid[current])

In [ ]:
proj = f"X{numx/10}c/{numy/10}c"
xrgrid = xr.DataArray(grid)
xrgrid.gmt.registration = 1
xrgrid.gmt.gtype = 0
fig = pygmt.Figure()
fig.grdimage(xrgrid, cmap='turbo', frame=True, projection=proj)
fig.plot(x=start[1], y=start[0], style='c0.15c', pen='0.5p,white', color='white')
fig.plot(x=end[1], y=end[0], style='c0.15c', pen='0.5p,white', color='black')
fig.colorbar()
fig.show(width=1000)

In [ ]:
xrgrid = xr.DataArray(grid)
xrgrid.gmt.registration = 1
xrgrid.gmt.gtype = 0
fig = pygmt.Figure()
fig.grdimage(xrgrid, cmap='turbo', frame=True, projection=proj, region=[-0.5,numx-0.5,-0.5,numy-0.5])
fig.plot(x=start[1], y=start[0], style='c0.15c', pen='0.5p,white', color='white')
fig.plot(x=end[1], y=end[0], style='c0.15c', pen='0.5p,white', color='black')
fig.plot(x=valid_path[:,1], y=valid_path[:,0], pen='1p,white')
fig.show(width=1400)
fig.savefig("not_quite_shortest_path.png")

In [ ]:
!open not_quite_shortest_path.png